<a href="https://colab.research.google.com/github/abnerlugo1/Power-query/blob/main/Delta_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import string
import re

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import brown
from nltk.corpus import stopwords

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

# Configuración warnings
# ==============================================================================
#import libarchive
#import pydot
#import cartopy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Lectura de datos
# ==============================================================================
file_path = '/content/base_mejora_enero.xlsx'  # base
df7 = pd.read_excel(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/base_mejora_enero.xlsx'

In [ ]:
df7.loc[df7["NPS"].isin([10, 9]), 'result'] = "promotor"
df7.loc[df7["NPS"].isin([7, 8]), 'result'] = "pasivo"
df7.loc[df7["NPS"].isin([1, 2, 3, 4, 5, 6]), 'result'] = "detractor"

In [ ]:
df7.loc[df7["EDS_F"].isin([5, 4]), 'result_EDS'] = "promotor"
df7.loc[df7["EDS_F"].isin([3]), 'result_EDS'] = "pasivo"
df7.loc[df7["EDS_F"].isin([1, 2]), 'result_EDS'] = "detractor"

In [ ]:
df7.isna().sum()

In [ ]:
print(df7['Mejora'])

In [ ]:
print(df7.groupby('NPS').size())

In [ ]:
# Distribución temporal
# ==============================================================================
fig, ax = plt.subplots(figsize=(9,4))

for NPS in df7.NPS.unique():
    df_temp = df7[df7['NPS'] == NPS].copy()
    df_temp['fecha_servicio'] = pd.to_datetime(df_temp['fecha_servicio'])
    df_temp = df_temp.groupby(df_temp['fecha_servicio']).size()
    df_temp.plot(label=NPS, ax=ax)

ax.set_title('NPS por mes')
ax.legend();

In [ ]:
import re
def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''

    # Se convierte todo el texto a minúsculas
    nuevo_texto = texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^\\_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]

    return(nuevo_texto)

test = "Esto es 1 ejemplo de l'limpieza de6 TEXTO  https://t.co/rnHPgyhx4Z @cienciadedatos #textmining"
test1= "12"
print(test)
print(limpiar_tokenizar(texto=test1))

In [ ]:
# Se aplica la función de quitar NAN
# ==============================================================================
cleaned_df = df7.dropna()

In [ ]:
# Se aplica la función de limpieza y tokenización
# ==============================================================================
df7['texto_tokenizado'] = df7['Mejora'].fillna('').apply(lambda x: limpiar_tokenizar(x))
df7[['Mejora', 'texto_tokenizado']].head()

In [ ]:
# Unnest de la columna texto_tokenizado
# ==============================================================================
tweets_tidy = df7.explode(column='texto_tokenizado')
# Drop the original 'Mejora' column as its content is now in 'texto_tokenizado' (soon to be 'token')
tweets_tidy = tweets_tidy.drop(columns='Mejora')
# Rename the exploded 'texto_tokenizado' column to 'token'
tweets_tidy = tweets_tidy.rename(columns={'texto_tokenizado':'token'})
tweets_tidy.head(3)

In [ ]:
# Palabras totales utilizadas por segmento
# ==============================================================================
print('--------------------------')
print('Palabras totales por servicio')
print('--------------------------')
tweets_tidy.groupby(by='EDS')['token'].count()

In [ ]:
# Longitud media y desviación de los coementarios de cada autor
# ==============================================================================
temp_df = pd.DataFrame(tweets_tidy.groupby(by = ["EDS_F", "token"])["token"].count())
temp_df = temp_df.rename(columns={'token': 'token_count'}) # Rename the count column
temp_df.reset_index().groupby("EDS_F")["token_count"].agg(['mean', 'std'])

In [ ]:
# Top 5 palabras más utilizadas por edad_cat
# ==============================================================================
tweets_tidy.groupby(['EDS'])['token'] \
 .count() \
 .reset_index(name='count') \
 .groupby('EDS') \
 .apply(lambda x: x.sort_values('count', ascending=False).head(5))

In [ ]:
nltk.download('stopwords')
# Obtención de listado de stopwords del español
# ==============================================================================
stop_words = list(stopwords.words('spanish'))
# Se añade la stoprword: amp, ax, ex
stop_words.extend(("por", "de", "para", "si", "vas", "mucha", "muchas"
,"merece", "pudiera","seco","viaje", "ahi", "ahí", "normal", "ir", "acelera"
, "además", "correo", "folio", "nunca", "buen", "bien", "buena", "sola"
, "solo", "bueno", "ser","uds", "vez", "dar", "así", "dio", "iban", "pedir"
,"cambio" , "cuito"  , "jajaja", "literla", "mandarme", "masculino", "memencanto"
, "mi", "dentro", "uds", "sé", "dónde", "dije", "el"))
print(stop_words[:10])

In [ ]:
# Filtrado para excluir stopwords
# ==============================================================================
tweets_tidy = tweets_tidy[~(tweets_tidy["token"].isin(stop_words))]

In [ ]:
# Top 10 palabras por autor (sin stopwords)
# ==============================================================================
fig, axs = plt.subplots(nrows=5, ncols=1,figsize=(6, 8))
for i, EDS in enumerate(tweets_tidy.EDS.unique()):
    df_temp = tweets_tidy[tweets_tidy.EDS == EDS]
    counts  = df_temp['token'].value_counts(ascending=False).head(10)
    counts.plot(kind='barh', color='firebrick', ax=axs[i])
    axs[i].invert_yaxis()
    axs[i].set_title(EDS)

fig.tight_layout()

In [ ]:
# Pivotado de datos
# ==============================================================================
tweets_pivot = tweets_tidy.groupby(["EDS","token"])["token"] \
                .agg(["count"]).reset_index() \
                .pivot(index = "token" , columns="EDS", values= "count")
tweets_pivot.columns.name = None

In [ ]:
# Test de correlación (coseno) por el uso y frecuencia de palabras
# ==============================================================================
from scipy.spatial.distance import cosine

def similitud_coseno(a,b):
    distancia = cosine(a,b)
    return 1-distancia

tweets_pivot.corr(method=similitud_coseno)

In [ ]:
# Gráfico de correlación
# ==============================================================================
f, ax = plt.subplots(figsize=(6, 4))
temp = tweets_pivot.dropna()
sns.regplot(
    x  = np.log(temp['Puede mejorar']),
    y  = np.log(temp['Cumple lo esperado']),
    scatter_kws =  {'alpha': 0.05},
    ax = ax
);
for i in np.random.choice(range(temp.shape[0]), 100):
    ax.annotate(
        text  = temp.index[i],
        xy    = (np.log(temp['Puede mejorar'][i]), np.log(temp['Cumple lo esperado'][i])),
        alpha = 0.7
    )

In [ ]:
# Número de palabras comunes
# ==============================================================================
palabras_promotor = set(tweets_tidy[tweets_tidy.EDS == 'Puede mejorar']['token'])
palabras_pasivo = set(tweets_tidy[tweets_tidy.EDS == 'Insatisfecho']['token'])
palabras_detractor = set(tweets_tidy[tweets_tidy.EDS == 'Supero expectativas']['token'])

print(f"Palabras comunes entre madrugada y tarde: {len(palabras_promotor.intersection(palabras_pasivo))}")
print(f"Palabras comunes entre madrugada y mañana: {len(palabras_detractor.intersection(palabras_pasivo))}")

In [ ]:
# Cálculo del log of odds ratio de cada palabra (pasivo vs detractor)
# ==============================================================================
# Pivotaje y despivotaje
tweets_pivot = tweets_tidy.groupby(["servicio","token"])["token"] \
                .agg(["count"]).reset_index() \
                .pivot(index = "token" , columns="EDS", values= "count")

tweets_pivot = tweets_pivot.fillna(value=0)
tweets_pivot.columns.name = None

tweets_unpivot = tweets_pivot.melt(value_name='n', var_name='EDS', ignore_index=False)
tweets_unpivot = tweets_unpivot.reset_index()

# Selección
tweets_unpivot = tweets_unpivot[tweets_unpivot.EDS.isin(['Insatisfecho', 'Supero expectativas'])]

# Se añade el total de palabras de cada autor
tweets_unpivot = tweets_unpivot.merge(
                    tweets_tidy.groupby('servicio')['token'].count().rename('N'),
                    how = 'left',
                    on  = 'servicio'
                 )

# Cálculo de odds y log of odds de cada palabra
tweets_logOdds = tweets_unpivot.copy()
tweets_logOdds['odds'] = (tweets_logOdds.n + 1) / (tweets_logOdds.N + 1)
tweets_logOdds = tweets_logOdds[['token', 'servicio', 'odds']] \
                    .pivot(index='token', columns='servicio', values='odds')
tweets_logOdds.columns.name = None

tweets_logOdds['log_odds']     = np.log(tweets_logOdds['Supero expectativas']/tweets_logOdds['Insatisfecho'])
tweets_logOdds['abs_log_odds'] = np.abs(tweets_logOdds.log_odds)

# Si el logaritmo de odds es mayor que cero, significa que es una palabra con
tweets_logOdds['autor_frecuente'] = np.where(tweets_logOdds.log_odds > 0,
                                              "Supero expectativas",
                                              "Insatisfecho"
                                    )

In [ ]:
print('-----------------------------------')
print('Top 10 palabras más diferenciadoras')
print('-----------------------------------')
tweets_logOdds.sort_values('abs_log_odds', ascending=False).head(10)

In [ ]:
# Top 15 palabras más características de cada autor
# ==============================================================================

top_30 = tweets_logOdds[['log_odds', 'abs_log_odds', 'autor_frecuente']] \
        .groupby('autor_frecuente') \
        .apply(lambda x: x.nlargest(15, columns='abs_log_odds').reset_index()) \
        .reset_index(drop=True) \
        .sort_values('log_odds')

f, ax = plt.subplots(figsize=(4, 7))
sns.barplot(
    x    = 'log_odds',
    y    = 'token',
    hue  = 'autor_frecuente',
    data = top_30,
    ax   = ax
)
ax.set_title('Top 15 palabras más características ')
ax.set_xlabel('log odds ratio (50  / _<28)');


In [ ]:
# Cálculo term-frecuency (tf)
# ==============================================================================
tf = tweets_tidy.copy()
# Número de veces que aparece cada término
tf = tf.groupby(["id", "token"])["token"].agg(["count"]).reset_index()
# Se añade una columna con el total de términos
tf['total_count'] = tf.groupby('id')['count'].transform(sum)
# Se calcula el tf
tf['tf'] = tf["count"] / tf["total_count"]
tf.sort_values(by = "tf").head(3)

In [ ]:
# Inverse document frequency
# ==============================================================================
idf = tweets_tidy.copy()
total_documents = idf["id"].drop_duplicates().count()
# Número de documentos (tweets) en los que aparece cada término
idf = idf.groupby(["token", "id"])["token"].agg(["count"]).reset_index()
idf['n_documentos'] = idf.groupby('token')['count'].transform(sum)
# Cálculo del idf
idf['idf'] = np.log(total_documents / idf['n_documentos'])
idf = idf[["token","n_documentos", "idf"]].drop_duplicates()
idf.sort_values(by="idf").head(3)

In [ ]:
# Term Frequency - Inverse Document Frequency
# ==============================================================================
tf_idf = pd.merge(left=tf, right=idf, on="token")
tf_idf["tf_idf"] = tf_idf["tf"] * tf_idf["idf"]
tf_idf.sort_values(by="id").head()

In [ ]:
# Reparto train y test
# ==============================================================================
# Filter tweets_tidy to only include the categories intended for classification
# Assuming the user wants to classify 'pasivo' vs 'detractor'
df_filtered = tweets_tidy[tweets_tidy.edad_cat.isin(['Edad_<_25', 'Edad_>_50'])].copy()

# X should be the features (e.g., tokens) and y should be the target labels
# Note: 'token' here is a single word from the exploded DataFrame. For actual classification,
# these tokens would usually be vectorized (e.g., TF-IDF) per document/comment ID.
datos_X = df_filtered['token']
datos_y = df_filtered['edad_cat']

X_train, X_test, y_train, y_test = train_test_split(
    datos_X,
    datos_y,
    test_size = 0.2,
    random_state = 123
)

In [ ]:
value, counts = np.unique(y_train, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))
value, counts = np.unique(y_test, return_counts=True)
print(dict(zip(value, 100 * counts / sum(counts))))

In [ ]:
# Creación de la matriz tf-idf
# ==============================================================================
tfidf_vectorizador = TfidfVectorizer(
                        tokenizer  = limpiar_tokenizar,
                        min_df     = 3,
                        stop_words = stop_words
                    )
tfidf_vectorizador.fit(X_train.dropna())

In [ ]:
tfidf_train = tfidf_vectorizador.transform(X_train.dropna())
tfidf_test  = tfidf_vectorizador.transform(X_test.dropna())

In [ ]:
print(f" Número de tokens creados: {len(tfidf_vectorizador.get_feature_names_out())}")
print(tfidf_vectorizador.get_feature_names_out()[:10])

In [ ]:
# Entrenamiento del modelo SVM
# ==============================================================================
modelo_svm_lineal = svm.SVC(kernel= "linear", C = 1.0)

# Filter y_train to match the non-null entries of X_train
y_train_filtered = y_train[X_train.notna()]

modelo_svm_lineal.fit(X=tfidf_train, y=y_train_filtered)

In [ ]:
# Grid de hiperparámetros
# ==============================================================================
param_grid = {'C': np.logspace(-5, 3, 10)}

# Búsqueda por validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = svm.SVC(kernel= "linear"),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = -1,
        cv         = 5,
        verbose    = 0,
        return_train_score = True
      )

# Se asigna el resultado a _ para que no se imprima por pantalla
_ = grid.fit(X = tfidf_train, y = y_train[X_train.notna()])

# Resultados del grid
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)

In [ ]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

modelo_final = grid.best_estimator_

In [ ]:
# Error predicciones test
# ==============================================================================
predicciones_test = modelo_final.predict(X=tfidf_test)

# Align y_test with the filtered X_test that was used for prediction
y_test_filtered_for_comparison = y_test[X_test.notna()]

print("-------------")
print("Error de test")
print("-------------")

print(f"Número de clasificaciones erróneas de un total de {tfidf_test.shape[0]} " \
      f"clasificaciones: {(y_test_filtered_for_comparison != predicciones_test).sum()}"
)
print(f"% de error: {100*(y_test_filtered_for_comparison != predicciones_test).mean()}")

print("")
print("-------------------")
print("Matriz de confusión")
print("-------------------")
# Get the unique classes to correctly label the confusion matrix
classes = sorted(y_test_filtered_for_comparison.unique())
pd.DataFrame(confusion_matrix(y_true = y_test_filtered_for_comparison, y_pred= predicciones_test),
             columns= [f"Predicted {c}" for c in classes],
             index = [f"Actual {c}" for c in classes])

In [ ]:
# Descarga lexicon sentimientos
# ==============================================================================
lexicon = pd.read_table(
            'https://raw.githubusercontent.com/fnielsen/afinn/master/afinn/data/AFINN-en-165.txt',
            names = ['termino', 'sentimiento']
          )
lexicon.head()

In [ ]:
# Sentimiento promedio
# ==============================================================================
tweets_sentimientos = pd.merge(
                            left     = tweets_tidy,
                            right    = lexicon,
                            left_on  = "token",
                            right_on = "termino",
                            how      = "inner"
                      )

tweets_sentimientos = tweets_sentimientos.drop(columns = "termino")

# The dataframe 'tweets_sentimientos' now contains the numeric 'sentimiento' column from the lexicon.
# No further sum/grouping is needed here if the intent is to analyze individual sentiments per 'edad_cat'
# in the next cell.
tweets_sentimientos.head()

In [ ]:
def perfil_sentimientos(df):
    print(autor)
    print("=" * 12)
    print(f"Positivos: {round(100 * np.mean(df.sentimiento > 0), 2)}")
    print(f"Neutros  : {round(100 * np.mean(df.sentimiento == 0), 2)}")
    print(f"Negativos: {round(100 * np.mean(df.sentimiento < 0), 2)}")
    print(" ")

for autor, df in tweets_sentimientos.groupby("edad_cat"):
    perfil_sentimientos(df)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))

for edad_cat in tweets_sentimientos.edad_cat.unique():
    df = tweets_sentimientos[tweets_sentimientos.edad_cat == edad_cat].copy()
    # Assuming 'fecha_servicio' is the date column to use for time series analysis
    # If 'Fecha encuesta' was intended, the column name must be consistent
    df['fecha_servicio'] = pd.to_datetime(df['fecha_servicio'])
    df = df.set_index("fecha_servicio")
    df = df[['sentimiento']].resample('1M').mean()
    ax.plot(df.index, df.sentimiento, label=edad_cat)

ax.set_title("Sentimiento promedio de los comentarios por mes")
ax.legend();